In [1]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [ ]:
resenha = pd.read_csv("../Dataset/imdb/imdb-pt-br.csv")

In [ ]:
resenha.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
treino, teste, classe_treino, classe_test =  train_test_split(resenha.text_pt,resenha.sentiment,random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
regressao_logistica = LogisticRegression()

In [ ]:
## regressao_logistica.fit(treino, classe_treino)

In [ ]:
print(resenha.text_pt[189])

In [ ]:
print(resenha.text_pt[49002])

In [ ]:
print(resenha.sentiment.value_counts())

In [ ]:
classificacao = resenha["sentiment"].replace(["neg","pos"],[0,1])

In [ ]:
resenha["classificacao"] =  classificacao

In [ ]:
resenha.head()

In [ ]:
resenha.tail()

In [ ]:
resenha.text_pt[49458]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vetorizar = CountVectorizer(lowercase=False)
texto = ["Assisti um filme bom","Assisti um filme ruim"]
bow = vetorizar.fit_transform(texto)
vetorizar.get_feature_names()

In [ ]:
matriz_esparsa =  pd.SparseDataFrame(bow,columns=vetorizar.get_feature_names())
matriz_esparsa

In [ ]:
vetorizar = CountVectorizer(lowercase=False,max_features=50)
bow = vetorizar.fit_transform(resenha.text_pt)
print(bow.shape)

In [ ]:
treino, teste, classe_treino, classe_test =  train_test_split(bow,resenha.classificacao,random_state=42)
regressao_logistica = LogisticRegression(solver='lbfgs')
regressao_logistica.fit(treino, classe_treino)
acuracia = regressao_logistica.score(teste,classe_test)
print(acuracia)

In [ ]:
def classificar_texto(texto,coluna_texto,coluna_classificacao):
    vetorizar = CountVectorizer(lowercase=False,max_features=50)
    bow = vetorizar.fit_transform(texto[coluna_texto])
    treino, teste, classe_treino, classe_test =  train_test_split(bow,texto[coluna_classificacao],random_state=42)
    regressao_logistica = LogisticRegression(solver='lbfgs')
    regressao_logistica.fit(treino, classe_treino)
    return regressao_logistica.score(teste,classe_test)

In [ ]:
print(classificar_texto(resenha,"text_pt","classificacao"))

In [ ]:
%matplotlib inline 

In [ ]:
from wordcloud import WordCloud

In [ ]:
todas_palavras = ''.join([texto for texto in resenha.text_pt])
nuvem_palavras = WordCloud(width=800,height=500,max_font_size=110,collocations=False).generate(todas_palavras)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15,12))
plt.imshow(nuvem_palavras,interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
def generate_img(dataframe,target,column_target):
    result = dataframe.query("sentiment == '{}'".format(target))
    todas_palavras = ''.join([texto for texto in result[column_target]])
    nuvem_palavras = WordCloud(width=800,height=500,max_font_size=110,collocations=False,background_color="White").generate(todas_palavras)
    plt.figure(figsize=(15,12))
    plt.imshow(nuvem_palavras,interpolation='bilinear')
    plt.axis("off")
    plt.show()

In [ ]:
generate_img(resenha,'neg','text_pt')

In [ ]:
generate_img(resenha,'pos','text_pt')

In [ ]:
import nltk
from nltk import tokenize

In [ ]:
__token = tokenize.WhitespaceTokenizer()

In [ ]:
phrase_tokenized = __token.tokenize(todas_palavras)

In [ ]:
frequency = nltk.FreqDist(phrase_tokenized)

In [ ]:
frequency

In [ ]:
## DataFrame 
frequency_df =  pd.DataFrame({"Palavra":list(frequency.keys()),"Frequencia":list(frequency.values())})
frequency_df.nlargest(columns="Frequencia",n=10)

In [2]:
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
plt.figure(figsize=(12,8))
ax = sns.barplot(data=frequency_df.nlargest(columns="Frequencia",n=10),x="Palavra",y="Frequencia",color="Green")
ax.set(ylabel="Contagem")
plt.show()

In [ ]:
def pareto(text_collection,column_target,amount,plot_color):
    all_words =  ''.join([texto for texto in text_collection[column_target]])
    __tokens = __token.tokenize(all_words)
    __frequency = nltk.FreqDist(__tokens)
    __df_frequency =  pd.DataFrame({"Word":list(__frequency.keys()),"Frequency":list(__frequency.values()),})
    __df_frequency =  __df_frequency.nlargest(columns="Frequency",n=amount)
    plt.figure(figsize=(12,8))
    __ax = sns.barplot(data=__df_frequency,x="Word",y="Frequency",color=plot_color)
    __ax.set(ylabel="Count")
    plt.show()

In [ ]:
pareto(resenha,"text_pt",10,"gray")

In [ ]:
palavras_irrelevantas = nltk.corpus.stopwords.words("portuguese")

In [ ]:
frase_processada = list()

In [ ]:
for texto in resenha.text_pt:
    nova_frase = list()
    palavras_texto = __token.tokenize(texto)
    for palavra in palavras_texto:
        if palavra not in palavras_irrelevantas:
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase))
    
resenha["tratamento_1"] = frase_processada

In [ ]:
resenha.head()

In [ ]:
classificar_texto(resenha,"tratamento_1","classificacao")

In [ ]:
pareto(resenha,"tratamento_1",10,"pink")

In [ ]:
palavras_irrelevantas